# Example notebook of how analysis can be scaled up

In [ ]:
import sys
sys.path.append('/home/jovyan/intake-aodn')
import intake_aodn
import intake

## Request a cluster with 2 CPU per worker, 4GB ram per worker, and allow to scale up to 256 workers

In [ ]:
from intake_aodn.utils import get_distributed_cluster
client = get_distributed_cluster(worker_cores=2,
                                 worker_memory=4,
                                 min_workers=1,
                                 max_workers=256)
client

# Open 20 years of MODIS Chl-a across all of Australia

In [ ]:
cat=intake_aodn.cat

In [ ]:
ds=cat.aodn_s3.MODIS_1d_chl_oc3(startdt='2002-01-01',
                                enddt='2021-06-30',
                                cropto=dict(longitude=slice(107.,160),
                                            latitude=slice(-9,-45))).to_dask()

In [ ]:
ds

In [ ]:
print(f'{ds.nbytes/1E9:.3f} GB')

## The data can be loaded dynamically from S3 into cluster memory for interactive viewing

In [ ]:
import hvplot.xarray
import geoviews as gv
from cartopy import crs as ccrs
gv.extension('bokeh')

In [ ]:
%%opts Overlay [width=900 height=600]

import warnings
warnings.filterwarnings("ignore")

(ds.chl_oc3.hvplot('longitude','latitude',groupby='time', dynamic=True, rasterize=True, cmap='magma',crs=ccrs.PlateCarree()).redim(chl_oc3=dict(range=(0, 1.0))))*\
(gv.feature.coastline(scale='10m')).opts(color='black')

# WA wide monthly mean

In [ ]:
ds_mean = ds.chl_oc3.groupby('time.month').mean()

In [ ]:
print(f'{ds_mean.nbytes/1E9:.3f} GB')

In [ ]:
ds_mean = ds_mean.load()

In [ ]:
import numpy as np
ds_mean = np.log(ds_mean)

In [ ]:
%%opts Overlay [width=900 height=600]

import warnings
warnings.filterwarnings("ignore")

(ds_mean.hvplot('longitude','latitude',groupby='month',dynamic=True, rasterize=True, cmap='magma',crs=ccrs.PlateCarree()).redim(chl_oc3=dict(range=(-3, 2))))*\
(gv.feature.coastline(scale='10m')).opts(color='black')

In [ ]:
client.cluster.shutdown()